In [149]:
import praw
import os
import configparser
import webbrowser
import random
import json
import pandas as pd
import numpy as np
import collections
os.chdir("C:/Users/macle/Desktop/UPC Masters/Semester 2/CI/Final Project")

In [128]:

config = configparser.ConfigParser()
config.read('secrets.ini')

reddit_client_id = config.get('reddit', 'client_id')
reddit_api_key = config.get('reddit', 'api_key')
r.set_oauth_app_info(client_id=reddit_client_id,
                      client_secret=reddit_api_key,
                      redirect_uri='http://cole-maclean.github.io/MAI-CI/')

In [129]:
url = r.get_authorize_url('uniqueKey', 'identity', True)
webbrowser.open(url)

True

In [130]:
access_information = r.get_access_information('B3w4W9k5xFbJpaLv8Xpuf-iQj34')
r.set_access_credentials(**access_information)

In [131]:
authenticated_user = r.get_me()
print(authenticated_user.name, authenticated_user.link_karma)

upcmaici 1


In [142]:
with open('reddit_data.json','r') as data_file:    
    reddit_data = json.load(data_file)
with open('scrapped_users.json','r') as data_file:    
    scrapped_users = json.load(data_file)
for i in range(10):
    print ("scrapping " + str(i) + "th subreddit")
    rand_submission = r.get_random_submission()
    print ("scrapping users from subreddit r/" + rand_submission.subreddit.display_name)
    if len(rand_submission.comments) >=3:
        rnd_comments = random.sample(rand_submission.comments,min(10,len(rand_submission.comments)))
        for comment in rnd_comments:
            if isinstance(comment, praw.objects.Comment):
                user = comment.author
                if user:
                    if user.name in scrapped_users:
                        print ('user ' + user.name + 'already scraped')
                    else:
                        scrapped_users.append(user.name)
                        for user_comment in user.get_comments(limit=None,_use_oauth=False):
                            body = user_comment.body.split()
                            if len(body) >= 10:
                                rand_words = random.sample(body,10)
                            else:
                                rand_words = body
                            reddit_data.append([user_comment.subreddit.display_name,
                                                                              user_comment.created_utc,rand_words])
                    
with open('reddit_data.json','w') as data_file:
    json.dump(reddit_data, data_file)
with open('scrapped_users.json','w') as data_file:
    json.dump(scrapped_users, data_file)

scrapping 0th subreddit
scrapping users from subreddit r/news
scrapping 1th subreddit
scrapping users from subreddit r/videos
scrapping 2th subreddit
scrapping users from subreddit r/grimfandango
scrapping 3th subreddit
scrapping users from subreddit r/streetwear
scrapping 4th subreddit
scrapping users from subreddit r/Foodforthought
scrapping 5th subreddit
scrapping users from subreddit r/hearthstone
scrapping 6th subreddit
scrapping users from subreddit r/aussievapers
scrapping 7th subreddit
scrapping users from subreddit r/technology
scrapping 8th subreddit
scrapping users from subreddit r/dirtykikpals
scrapping 9th subreddit
scrapping users from subreddit r/futurama


In [147]:
df = pd.DataFrame(reddit_data,columns=['subreddit','utc_stamp','rnd_words'])
df.head()

,subreddit,utc_stamp,rnd_words
0,breathinginformation,1479531643,"[beat, Griffon, the, ball, when, in, Peggle, d..."
1,baseball,1479531569,"[Hopefully, both, deals.]"
2,baseball,1479530961,"[Rodon, and, Gonzalez, 1-2., Shields, 3.]"
3,baseball,1479530947,"[I, count, Rodon, and, Miguel, Gonzalez, befor..."
4,breathinginformation,1479527248,"[Ryan, dropped, old, in, school., knife, Wow, ..."


In [151]:
f = {'subreddit':['count'], 'utc_stamp':[np.min,np.max],'rnd_words':['count']}
grouped = df.groupby('subreddit').agg(f)
grouped

rnd_words   utc_stamp             subreddit
                          count        amin        amax     count
subreddit                                                        
112263Hulu                    6  1460064617  1460072870         6
1P_LSD                        1  1478711407  1478711407         1
2007scape                     8  1400730277  1463604421         8
30ROCK                       25  1416533696  1478398979        25
3dshacks                      4  1471084549  1471085173         4
3kliksphilip                  9  1461453823  1463766346         9
4PanelCringe                  6  1470073248  1477963749         6
4chan                        88  1460572878  1479556370        88
60fpsporn                     3  1468554853  1471113878         3
7daystodie                    3  1470258272  1472933673         3
AFOL                          2  1382629970  1411052347         2
AMA                           4  1469522136  1479008046         4
ANormalDayInRussia            2  1473639805  1478602789         2
ARK                           1  1478492964  1478492964         1
ASKGAMING                     1  1428875800  1428875800         1
AVoid5                        3  1474043614  1474638336         3
AbandonedPorn                 1  1428265346  1428265346         1
AbigailRatchford              2  1477017887  1477127832         2
AccidentalComedy              4  1445914976  1461359405         4
AccidentalRenaissance         1  1470662908  1470662908         1
ActLikeYouBelong              1  1446766235  1446766235         1
ActionFigures                 3  1436461249  1449777388         3
Acura                         2  1474800133  1475930064         2
AdPorn                        2  1472869452  1472930110         2
AdamCarolla                  75  1440547564  1477667354        75
Advice                        7  1399140126  1469958776         7
AdviceAnimals               655  1365287327  1479540392       655
AgainstHateSubreddits        10  1479338219  1479547211        10
AlienBlue                     8  1413662043  1454971553         8
Alonetv                       1  1464478381  1464478381         1
...                         ...         ...         ...       ...
wolframalpha                  1  1465683240  1465683240         1
womensstreetwear              1  1479175377  1479175377         1
woodworking                  27  1420194513  1479494328        27
workaholics                   1  1454872111  1454872111         1
workgonewild                  1  1473130725  1473130725         1
worldbuilding               110  1476496128  1479335429       110
worldnews                   560  1365527198  1479547893       560
worldofpvp                    1  1479239165  1479239165         1
worldpolitics                 6  1418314505  1479036820         6
worstof                       3  1479391993  1479524587         3
wow                         393  1447443703  1479530491       393
wowthissubexists              5  1469206787  1475325921         5
wrestlingmemorabilia          2  1477860075  1478647762         2
writing                       7  1435594735  1478886112         7
wyoming                       2  1392838106  1392838364         2
xbox                          4  1417909974  1418339650         4
xbox360                       4  1417846570  1435425204         4
xboxlive                      1  1420652212  1420652212         1
xboxone                       9  1408432928  1466632538         9
xkcd                         34  1456423758  1479153835        34
xmen                          3  1461594439  1461626247         3
ynab                          5  1454178473  1454334281         5
youdontsurf                   3  1445975996  1478825042         3
yourmomshousepodcast          1  1473707355  1473707355         1
youseeingthisshit             1  1478850740  1478850740         1
youtube                       8  1444223037  1476419596         8
youtubehaiku                 15  1410396061  1477532003       

In [133]:
all_subreddits = []
list_of_words = []
for key,data in reddit_data.items():
    for user_comment in data:
        all_subreddits.append(user_comment[0])
        for word in user_comment[2]:
            list_of_words.append(word)
subs = set(all_subreddits)
bag_of_words = set(list_of_words)

In [135]:
len(reddit_data.keys())

741